# LEAP Plotting in Jupyter

Easy access to visualization makes for a smooth algorithm development cycle. LEAP offers components to assist with plotting algorithms and problems with `matplotlib` and Jupyter.

The key to live animation in Jupyter is to use the `%matplotlib notebook` magic:

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt

If you use `%matplotlib inline` instead, you'll end up with a static image.

## Live BSF Curves

We usually visualize things in LEAP the same way we measure anything: with a **probe operator**.

When `PopulationPlotProbe` is inserted into an operator pipeline, it captures the whole population at each generation and plots a statistic from it.  By default, it measures the **best-of-generation fitness**.

Next we'll create the probe operator.  It will automatically set up a figure and prepare to plot to it.  The probe will automatically scale the axes to ensure that the entire curve stays within view, but here we'llmanually set the initial `ylim` bounds to add more space:

In [2]:
from leap_ec import core
from leap_ec.probe import PopulationPlotProbe

# The probe needs access to a context object so that it can read the global generation counter
plot_probe = PopulationPlotProbe(core.context, ylim=(0, 70))

ModuleNotFoundError: No module named 'leap_ec'

Not much is happening yet.  We've created the probe and attached it to a figure—but it **still needs an algorithm!**

Let's run a simple generational, real-valued EA to solve the `Spheroid` function.  We want the population to pass through the probe at each iteration, so that it can collect data for the y-coordinate.  We achieve this by inserting it at the end of the operator pipeline:
 
 ```Python
 pipeline=[
           ...,
           ops.pool(size=pop_size),
           plot_probe
       ]
 ```

We put the probe right after `ops.pool` because the probe needs to receive the **population** all at once, instead of of one individual at a time (`ops.pool` takes a stream of generated individuals and makes a population from it).

In [ ]:
from leap_ec.algorithm import generational_ea
from leap_ec import core, ops, real_problems

l=10
pop_size=10
ea = generational_ea(generations=100, pop_size=pop_size,
                     problem=real_problems.SpheroidProblem(maximize=False),
                     
                     representation=core.Representation(
                         individual_cls=core.Individual,
                         decoder=core.IdentityDecoder(),
                         initialize=core.create_real_vector(bounds=[[-5.12, 5.12]] * l)
                     ),

                     pipeline=[
                         ops.tournament,
                         ops.clone,
                         ops.mutate_gaussian(std=0.5),
                         ops.evaluate,
                         ops.pool(size=pop_size),
                         plot_probe  # Insert the probe into the pipeline like so
                     ])
list(ea);

Scroll up to the previous cell while the algorithm is running—the live results get placed there, thanks to the callbacks on `plot_probe`!

## Viewing Fitness Functions

When dealing with basic benchmark functions, we very often want to take a look at the problem we're solving (at the very least to make sure we're solving the right one!).

To save you typing with `matplotlib`, LEAP offers convenience methods for plotting real-valued fitness surfaces.

Here's how we'd view a surface plot for the `SpheroidProblem`:

In [ ]:
from leap_ec import real_problems as real

real.plot_2d_problem(real.SpheroidProblem(), xlim=(-5.12, 5.12), ylim=(-5.12, 5.12));

You can also plot a function's fitness contours.

This time we'll explicitly set up a `matplotlib` figure and tell `plot_2d_problem` to plot to its axes (with the `ax=plt.gca()` parameter).  This allows us to configure subplots and arrange them side-by-side (using standard `matplotlib` foo:

In [ ]:
problem = real_problems.SchwefelProblem()
bounds = problem.bounds

fig = plt.figure(figsize=(8, 3))

plt.subplot(121, projection='3d')
real_problems.plot_2d_problem(problem, xlim=bounds, ylim=bounds, ax=plt.gca())

plt.subplot(122)
real_problems.plot_2d_problem(problem, kind='contour', xlim=bounds, ylim=bounds, ax=plt.gca());

## Viewing Population Trajectories



In [ ]:
problem = real_problems.SchwefelProblem()
bounds = problem.bounds

In [ ]:
from leap_ec.probe import PlotTrajectoryProbe

plt.figure(figsize=(8, 3))  # Setup a figure to plot to

plt.subplot(121)
trajectory_probe = PlotTrajectoryProbe(core.context, contours=problem, ax=plt.gca(), xlim=bounds, ylim=bounds)

plt.subplot(122)
fitness_probe = PopulationPlotProbe(core.context, ax=plt.gca())

In [ ]:
from leap_ec.algorithm import generational_ea
from leap_ec import core, ops, real_problems

l=10
pop_size=10
ea = generational_ea(generations=100, pop_size=pop_size,
                     problem=real_problems.SpheroidProblem(maximize=False),
                     
                     representation=core.Representation(
                         decoder=core.IdentityDecoder(),
                         initialize=core.create_real_vector(bounds=[problem.bounds] * l)
                     ),

                     pipeline=[
                         ops.tournament,
                         ops.clone,
                         ops.mutate_gaussian(std=50, hard_bounds=problem.bounds),
                         ops.evaluate,
                         ops.pool(size=pop_size),
                         trajectory_probe,  # Insert the probes into the pipeline like so
                         fitness_probe
                     ])
list(ea);